In [ ]:
import torch
from model.model1 import Net
import pandas as pd
import random

In [ ]:
# Load Training dataset
df = pd.read_csv('data/cleaned_train.csv')

In [ ]:
# Divide between train and test data
train = df[:800]
test = df[800:]
# Divide between x,y 
train_x = torch.tensor(train.iloc[:,3:].values).float()
train_y = torch.tensor(train.iloc[:,2].values).float()
test_x = torch.tensor(test.iloc[:,3:].values).float()
test_y = torch.tensor(test.iloc[:,2].values).float()


In [ ]:
# Create the network
net = Net(inputs=17,outputs=1,dropout_prob=0.01)
# Move all to gpu
if torch.cuda.is_available():
   net = net.to("cuda:0")
   train_x = train_x.to("cuda:0")
   train_y = train_y.to("cuda:0")
   test_x = test_x.to("cuda:0")
   test_y = test_y.to("cuda:0")



In [ ]:
# Prepare the network for training
epochs = 3000
batch_size = 10
# Initialize loss function
loss_fn = torch.nn.BCELoss()
# Initialize optimizer
optm = torch.optim.Adam(params=net.parameters(),lr=1e-2)
scheduler1 = torch.optim.lr_scheduler.LinearLR(optm,1,0.1,epochs)


In [ ]:
# Divide tensors into batch sizes
train_x = train_x.split(batch_size);
train_y = train_y.split(batch_size);

In [ ]:
losses = []
accuracies = []
num_batches = 50
test_y = test_y.reshape((test_y.shape[0],1))
for epoch in range(epochs):
    net.train()
    for i in range(num_batches):
        indx = random.randint(0,len(train_x)-1)
        x = train_x[indx]
        y = train_y[indx]
        net_out = net.forward(x,x.size()[0])
        #print(all(i == net_out[0] for i in net_out))
        loss = loss_fn.forward(net_out,y.reshape(y.size()[0],1))
        optm.zero_grad()
        loss.backward()
        optm.step()
    scheduler1.step()
    with torch.no_grad():
        net.eval()
        net_out = net.forward(test_x,test_x.size()[0])
        loss = loss_fn.forward(net_out,test_y.reshape(test_y.size()[0],1))
        predictions = net_out.round()
        accuracy = (predictions - test_y).abs().sum()/test_y.size()[0]
        losses.append(loss)
        accuracies.append(accuracies)
        print(f'EPOCH: {epoch}, Loss: {loss}, Accuracy: {accuracy}')
